In [ ]:
import cv2
import time
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import warnings
import os
from tiki.mini import TikiMini
from ultralytics import YOLO

# 분리된 모듈 import
from utils.config import (
    BIRD_WIDTH, BIRD_HEIGHT, JSON_PATH
)
from utils.pid_controller import PIDController
from utils.server_communication import send_to_server, send_dashboard_image, startSend, wait_for_json_file
from utils.lane_detection import detect_lane_center, make_panel
from utils.pothole_detection import check_and_handle_pothole
from utils.image_utils import frame_to_bytes_drive, frame_to_bytes_yolo
from utils.aruco_detection import ArUcoDetector
from utils.yolo_detection import YOLODetector
from utils.camera_utils import init_camera, init_perspective_transform
from utils.driving_control import control_motors

warnings.filterwarnings('ignore', category=UserWarning)
os.environ['GST_DEBUG'] = '0'

##############################################
# 1. YOLOv8 학습 모델 로드 (가장 먼저 실행)
##############################################
model = YOLO("./best4.pt")  # 🔥 학습한 best.pt 경로 확인
print("[YOLO] 모델 로드 완료")


##############################################

##############################################
# 2. ArUco 설정
##############################################
aruco_detector = ArUcoDetector()

##############################################
# 3. 로봇 초기화
##############################################
tiki = TikiMini()
tiki.set_motor_mode(tiki.MOTOR_MODE_PWM)
# 안전: 초기화 직후 모터 정지
try:
    tiki.stop()
except Exception:
    pass
print("로봇 초기화 완료 (모터 정지)")

# PID 제어
pid = PIDController()

##############################################
# 4. 단일 카메라 파이프라인 (주행 + YOLO 공용)
##############################################
cap = init_camera()

##############################################
# 5. Jupyter용 두 화면 위젯 (주행 / 객체인식)
##############################################
drive_widget = widgets.Image(
    format='jpeg',
    layout=widgets.Layout(width='640px', height='480px')
)
yolo_widget = widgets.Image(
    format='jpeg',
    layout=widgets.Layout(width='640px', height='480px')
)

display(widgets.HBox([drive_widget, yolo_widget]))

##############################################
# 6. Bird's-Eye View 설정 (주행용)
##############################################
M = init_perspective_transform()

##############################################
# 6-1. YOLO 감지기 초기화
##############################################
yolo_detector = YOLODetector(model, aruco_detector)
##############################################
# 1-1. A3R8.json 파일 대기 (파일이 생성될 때까지 무한 대기)
##############################################
test_data = wait_for_json_file()

##############################################
# 1-2. 시작 통신 전송 (startdashboard 로직)
startSend()
##############################################
# 7. 메인 루프 (단일 카메라 → 주행 + YOLO 두 화면 출력)
##############################################
print("[START] 주행 + YOLO + ArUco 시스템 실행")

frame_idx = 0
annotated = None  # 마지막 YOLO 결과 프레임 보관

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            time.sleep(8)
            continue

        frame_idx += 1

        # 공통 원본 프레임 (카메라 기준 회전 보정)
        frame = cv2.flip(frame, -1)

        # 주행용 처리를 위해 320x240으로 다운스케일한 프레임 사용
        frame_small = cv2.resize(frame, (BIRD_WIDTH, BIRD_HEIGHT))


        # ----- (1) 주행용 버드뷰 / 차선 인식 -----
        bird = cv2.warpPerspective(frame_small, M, (BIRD_WIDTH, BIRD_HEIGHT))
        center_x, is_horizontal_bar, dbg = detect_lane_center(bird)
        view = bird.copy()

        # ----- (2) 객체 인식 (YOLO + ArUco) : N프레임마다만 수행 -----
        frame_for_aruco = frame.copy()

        # (A) ArUco 포인트 감지 (원본 해상도 사용)
        corners, ids, _ = aruco_detector.detect(frame_for_aruco)
        aruco_detector.process_markers(frame_for_aruco, ids, corners)

        # (B) YOLO 객체 감지
        annotated, detected_counts = yolo_detector.detect_objects(frame, frame_idx)

        # ----- (3) 두 화면을 Jupyter 위젯으로 출력 -----
        drive_widget.value = frame_to_bytes_drive(panel)
        if annotated is not None and frame_idx % 2 == 0:
            yolo_widget.value = frame_to_bytes_yolo(annotated)

        # 주행 제어
        control_motors(tiki, center_x, is_horizontal_bar, pid)

        panel = make_panel(view, dbg)
                    
        # ----- 포트홀 감지 (적응형) -----
        check_and_handle_pothole(bird, tiki)


        time.sleep(0.02)

except KeyboardInterrupt:
    tiki.stop()
    cap.release()
    cv2.destroyAllWindows()
    print("[END] 프로그램 종료")


로봇 초기화 완료 (모터 정지)
GST_ARGUS: Cleaning up
CONSUMER: Done Success
GST_ARGUS: Done Success
GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3264 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3264 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 12

[ WARN:0@298.703] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0@298.708] global cap_gstreamer.cpp:2152 setProperty OpenCV | GStreamer warning: GStreamer: unhandled property


[START] 주행 + YOLO + ArUco 시스템 실행
[POTHOLE] 감지: white_frac=0.000 baseline=0.000 → 회피 수행
[AVOID] 포트홀 회피 시작 (좌회전)
[AVOID] 회피 완료

0: 320x416 (no detections), 72.0ms
Speed: 6.9ms preprocess, 72.0ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 74.7ms
Speed: 6.9ms preprocess, 74.7ms inference, 3.4ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 77.6ms
Speed: 7.3ms preprocess, 77.6ms inference, 2.8ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 74.8ms
Speed: 7.0ms preprocess, 74.8ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 74.9ms
Speed: 6.7ms preprocess, 74.9ms inference, 3.1ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 75.1ms
Speed: 6.9ms preprocess, 75.1ms inference, 2.7ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 74.9ms
Speed: 7.0ms preprocess, 74.9ms inference